In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from matplotlib import font_manager
import seaborn as sns
from matplotlib.font_manager import FontProperties  

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# 读取数据
train = pd.read_csv('/root/autodl-fs/data/train_revise+45缩减到100特征 数量1000个 去掉三列和Name.csv')

# 分离特征和标签
X = train.drop(['senolytic'], axis=1)
y = train['senolytic']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. 使用SMOTE进行过采样
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 2. 添加轻微噪声扰动
def add_noise(X, noise_factor=0.01):
    """
    向数据添加轻微噪声扰动
    noise_factor：噪声的强度（默认为0.01，表示噪声相对于原始数据的比例）
    """
    noise = np.random.normal(0, noise_factor, X.shape)
    return X + noise

X_train_smote_noisy = add_noise(X_train_smote)

# 3. 使用MixUp进行数据增强
def mixup(X, y, alpha=0.2):
    """
    使用MixUp生成样本
    alpha：Beta分布的参数（控制插值的强度）
    """
    # 从Beta分布采样
    lam = np.random.beta(alpha, alpha, size=X.shape[0])
    
    # 随机选择两个样本进行插值
    indices = np.random.permutation(X.shape[0])
    
    X_mix = lam.reshape(-1, 1) * X + (1 - lam.reshape(-1, 1)) * X.iloc[indices]
    y_mix = lam * y + (1 - lam) * y.iloc[indices]
    
    return X_mix, y_mix

X_train_final, y_train_final = mixup(X_train_smote_noisy, y_train_smote)

# 打印增强后的数据集信息
print(f"原始训练集样本数：{len(X_train)}")
print(f"SMOTE后训练集样本数：{len(X_train_smote)}")
print(f"加入噪声后训练集样本数：{len(X_train_smote_noisy)}")
print(f"MixUp增强后训练集样本数：{len(X_train_final)}")

原始训练集样本数：800
SMOTE后训练集样本数：1254
加入噪声后训练集样本数：1254
MixUp增强后训练集样本数：1254


In [3]:
# 保存增强后的数据集（SMOTE + 噪声扰动 + MixUp）
enhanced_data = pd.DataFrame(X_train_final)  # 混合后的特征数据
enhanced_data['senolytic'] = y_train_final  # 添加标签列

# 保存为CSV文件
enhanced_data.to_csv('/root/autodl-fs/data/enhanced_train_data.csv', index=False)

# 确认保存成功
print("增强后的数据集已保存：/root/autodl-fs/data/enhanced_train_data.csv")

/tmp/ipykernel_1257/2677801922.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  enhanced_data['senolytic'] = y_train_final  # 添加标签列


增强后的数据集已保存：/root/autodl-fs/data/enhanced_train_data.csv
